# Tarea 9

A continuación se crearán modelos para poder predecir la proporción de propina de un viaje. Además, dichos modelos se implementaran a través de búsquedas paralelas y secuenciales, con el objetivo de evaluar sus tiempos de ejecución. Utilizaremos funciones de scikit learn, Dask-ML  y el decorador **delayed** para ejecutar en paralelo.


Con la información resultante se responderán las siguientes preguntas: 

**1.- ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?**

**2.- ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?**

______________________________________________________________________________________________________

Primeramente se importan las librerías

In [175]:
import time
import numpy as np
import dask.dataframe as dd
from dask import delayed
from dask_ml.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import Ridge

Se cargan los datos y se exploran

In [176]:
df = dd.read_csv("trips.csv", assume_missing=True)
df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1.0,1.0,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1.0,1.0,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1.0,1.0,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1.0,1.0,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1.0,1.0,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


Se adiciona la proporción de propina por viaje como variable con el título de columna: "tip_percentage"

In [177]:
df["tip_percentage"] = (df["tip_amount"]/df["fare_amount"])

Se da el formato faltante a las variables de interés

In [178]:
df['tpep_pickup_datetime'] = dd.to_datetime(df['tpep_pickup_datetime'])
df['hour'] = df['tpep_pickup_datetime'].dt.hour

Se limpian los datos, eliminando las filas con datos faltantes

In [179]:
df=df.dropna()

Dado que la variable que se quiere pronosticar es continua, se utilizarán regresiones lineales. Se selecciona la variable dependiente("y") y las variables independientes("x").  

In [180]:
y = df["tip_amount"]
x = df.drop(df.columns[[0,3, 4, 5, 6]], axis=1)
x.head()

,fare_amount,passenger_count,trip_distance,tip_percentage,hour
0,22.0,1.0,6.90,0.209091,1
1,9.0,1.0,1.81,0.000000,23
2,7.5,1.0,0.96,0.133333,15
3,8.5,1.0,1.90,0.117647,8
4,7.5,1.0,1.00,0.221333,12


Se crean los sets de entrenamiento y prueba

In [181]:
x_train, x_test, y_train, y_test = train_test_split(x.compute(), y.compute(), test_size=0.2, random_state=42)

Se instancian la regresión lineal y el buscador grid

In [182]:
# Setup the hyperparameter grid
param_grid = {'C':np.linspace(0,1,30),'fit_intercept' :[True,False]}

# Instantiate a linear regression: linear_reg
linear_reg = LinearRegression()

# Instantiate the GridSearchCV object: linear_reg_cv
linear_reg_cv = GridSearchCV(linear_reg, param_grid, cv=5)

## Pregunta 1: ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Se entrena el modelo y evalúa su tiempo de ejecución bajo una búsqueda secuencial

In [183]:
# Call time.time()
t0_a = time.time()

# Fit it to the data
fit_train = linear_reg_cv.fit(x_train.values, y_train.values)

# Call time.time()
t1_a = time.time()

# Print the execution time
print((t1_a-t0_a)/60)

/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_mode

0.612762987613678


Se agrega el decorador **delayed** para ejecutar en paralelo

In [186]:
def paralel_train(x_train_, y_train_):
    return fit_train(x_train_, y_train_)

delayed_fit_train = delayed(paralel_train)

Se entrena el modelo y evalúa su tiempo de ejecución bajo una búsqueda en paralelo

In [191]:
# Call time.time()
t0_b = time.time()

# Fit it to the data
delayed_fit_train(x_train.values, y_train.values)

# Call time.time()
t1_b = time.time()

# Print the execution time
print((t1_b-t0_b)/60)

2.1870930989583334e-05


Se evalúa el modelo

In [192]:
linear_reg_cv.best_estimator_

LinearRegression(C=0.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)

In [206]:
linear_reg_cv.score(x_test.values, y_test.values)

7.253391843393149

In [207]:
print("Tuned Linear Regression Accuracy: {}".format(linear_reg_cv.best_score_))

Tuned Linear Regression Accuracy: 7.095717577487766


#### Resultado: 

La búsqueda secuencial tardó 0.6127 minutos, mientras que la búsqueda en paralelo duró .000021 minutos. Por lo que se concluye que la busqueda en paralelo es más eficiente en cuanto a tiempo. 

## Pregunta 2: ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

Se instancian la regresión lineal y el buscador grid

In [203]:
lr = LinearRegression()

param_grid_ = {'C' :np.linspace(0,1,30),'fit_intercept' :[True,False]}

from dask_ml.model_selection import GridSearchCV as GridSearchCV_ml

# Instantiate the GridSearchCV object: ridge_reg_cv
linear_reg_cv_ml = GridSearchCV_ml(lr, param_grid_, n_jobs=4, cv=5)

Se entrena el modelo y evalúa su tiempo de ejecución bajo una búsqueda con Dask ML

In [205]:
# Call time.time()
t0_c = time.time()

# Fit it to the data
fit_train_ml = linear_reg_cv_ml.fit(x_train.values, y_train.values)

# Call time.time()
t1_c = time.time()

# Print the execution time
print((t1_c-t0_c)/60)

/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_model/glm.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  'lamduh': 1 / self.C}
/usr/local/lib/python3.5/site-packages/dask_ml/linear_mode

13.71938523054123


Se evalúa el modelo

In [208]:
linear_reg_cv_ml.best_estimator_

LinearRegression(C=0.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)

In [209]:
linear_reg_cv_ml.score(x_test.values, y_test.values)

7.253391843393149

In [214]:
print("Tuned Linear Regression ML Accuracy: {}".format(linear_reg_cv_ml.best_score_))

Tuned Linear Regression ML Accuracy: 7.095717577487766


### Resultado

El tiempo de ejecución de búsqueda con Dask ML fué de 13.71 minutos. A comparación de los modelos anteriores, este fué el peor tiempo de ejecución.